In [55]:
import requests
import json
import pandas as pd
from tqdm import tqdm

In [6]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
}
params = {
    "limit": "48", # limit number of items each page
    "include": "advertisement",
    "aggregation": "2",
    "category": "1846",
    #"page": "1", # page number
    "urlkey": "laptop-may-vi-tinh-linh-kien" # product category
}
new_api_request_url = "https://tiki.vn/api/v2/products" # new api from tiki developer
old_api_request_url = "https://tiki.vn/api/personalish/v1/blocks/listings" # current api from tiki webpage
#pageNum = 1
product_id_list = []
df_id = pd.DataFrame([], columns=["id"])
# old api page: 2000 - 209   2000 - 209
# old api no page: 47376 - 988
# new-api-page : 2000 - 198  2000 - 199  2000 - 240 
# new-api-no page:  10080 - 211    8400 - 176

<b>Fetch Product ID Data<b>

In [7]:
while True:       
    '''With the page number param available, data return only 2000 items -> I will block it to get as much data as possible''' 
    #print("Crawl page number: ", pageNum)
    #params["page"] = pageNum
    response = requests.get(new_api_request_url, headers=headers, params=params)
    if response.status_code == 200:
        #print("Crawling data successfully")
        #products = json.loads(response.text)["data"]
        for product in response.json()["data"]:
            id = str(product["id"])
            product_id_list.append(id)
    else:
        print(f"Error, status code = {response.status_code}")
        break

    # Divided each batch with maximum 2000 items 
    if(len(product_id_list) % 2000 == 0):
        print("Reach 2000 items")
        df_temp = pd.DataFrame(product_id_list, columns=["id"])
        df_id = pd.concat([df_id, df_temp], ignore_index=True)
        # Truncate all the list
        del product_id_list[:]
        print(len(product_id_list))
        
    #pageNum+=1

Reach 2000 items
0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
print(len(product_id_list))
df_id.shape

5472


(6000, 1)

In [9]:
# Fill out the list that not reach 2000 items
df_temp = pd.DataFrame(product_id_list, columns=["id"])
df_id = pd.concat([df_id, df_temp], ignore_index=True)
# Truncate all the list
del product_id_list[:]
print(len(product_id_list))
df_id.shape

0


(11472, 1)

In [10]:
df_id.head()

,id
0,134561311
1,38323087
2,26314426
3,8608925
4,74975883


In [11]:
list_temp = df_id["id"].values.tolist()

11472


In [51]:
print(len(list_temp))

11472


<b> Fetch Product Details Data <b>

In [150]:
url = "https://tiki.vn/api/v2/products/{}"
pars = {"platform": "web", 
        "spid": "124939771"
}
product_detail_list = []
product_json_list = []
df_product = pd.DataFrame([])

In [149]:
flatten_field = ["id", "name", "sku", "price", "list_price", "discount", "discount_rate", 
                                                    "inventory_status", "stock_item_max_sale_qty", "stock_item_qty", "qty_sold", "brand_id", "branch_name"]
def adjust(product):
    e = json.loads(product)
    for field in flatten_field:
        if field in e:
            e[field] = json.dumps(e[field], ensure_ascii=False).replace('\n','')
    return e

In [151]:
i = 0
for id in tqdm(list_temp, total=len(list_temp)):
    response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=pars)
    if(response.status_code==200):
        # Extract value attributes
        # product["id"] = response.json()['id']
        # product["name"] = response.json()['name']
        # product["sku"] = response.json()['sku']
        # product["price"] = response.json()['price']
        # product["list_price"] = response.json()['list_price']
        # product["discount"] = response.json()['discount']
        # product["discount_rate"] = response.json()['discount_rate']
        # product["inventory_status"] = response.json()['inventory_status']
        # product["stock_item_max_sale_qty"] = response.json()['stock_item']['max_sale_qty']
        # product["stock_item_qty"] = response.json()['stock_item']['qty']
        # product["qty_sold"] = response.json()['quantity_sold']['value']
        # product["brand_id"] = response.json()['brand']['id']
        # product["brand_name"] = response.json()['brand']['name']
        # product_detail_list.append(product)         
        product_detail_list.append(response.text)
        i+=1
    else:
        print(f"Error, status code = {response.status_code}")
        pass

    # Divided each batch with maximum 2000 items 
    if(len(product_detail_list) == 20):
        product_json_list = [adjust(p) for p in product_detail_list]
        print("Reach 2000 items")
        df_temp = pd.DataFrame.from_dict(product_json_list)
        df_product = pd.concat([df_product, df_temp], ignore_index=True)
        # Truncate all the list
        del product_detail_list[:]
        del product_json_list[:]
        print(len(product_detail_list), len(product_json_list))
    if i==50: break
        

  0%|          | 0/11472 [00:00<?, ?it/s]

  0%|          | 20/11472 [00:06<1:17:25,  2.47it/s]

Reach 2000 items
0 0


  0%|          | 40/11472 [00:14<1:03:19,  3.01it/s]

Reach 2000 items
0 0


  0%|          | 49/11472 [00:17<1:09:10,  2.75it/s]


In [152]:
print(len(product_detail_list), len(product_json_list))
df_product.shape

10 0


(40, 70)

In [153]:
# Fill out the list that not reach 2000 items
product_json_list = [adjust(p) for p in product_detail_list]
df_temp = pd.DataFrame.from_dict(product_json_list)
df_product = pd.concat([df_product, df_temp], ignore_index=True)
# Truncate all the list
del product_detail_list[:]
del product_json_list[:]
print(len(product_detail_list), len(product_json_list))
df_product.shape

0 0


(50, 70)

In [154]:
df_product.head()

,id,master_id,sku,name,url_key,url_path,short_url,type,book_cover,short_description,...,inventory,warranty_info,return_and_exchange_policy,is_tier_pricing_available,is_tier_pricing_eligible,asa_cashback_widget,benefits,price_comparison,video_url,best_price_guaranteed
0,134561311,134561311,"""8367091212771""","""USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập ...",usb-3-0-sandisk-ultra-flair-cz73-64gb-hang-nha...,usb-3-0-sandisk-ultra-flair-cz73-64gb-hang-nha...,https://tiki.vn/product-p134561311.html?spid=9...,configurable,None,Phần mềm bảo vệ và chống truy cập trái phép Se...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '5 Nă...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 212, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN
1,38323087,38323087,"""9386902161867""","""(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passp...",o-cung-di-dong-wd-my-passport-model-2019-hang-...,o-cung-di-dong-wd-my-passport-model-2019-hang-...,https://tiki.vn/product-p38323087.html?spid=53...,configurable,None,Thông số kỹ thuật WD My Passport màu đen mode...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '36 T...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 212, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN
2,26314426,26314426,"""9365508570174""","""Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu""",day-cap-mang-cat6-bam-san-2-dau-p26314426,day-cap-mang-cat6-bam-san-2-dau-p26314426.html...,https://tiki.vn/product-p26314426.html?spid=14...,configurable,None,+ Cáp mạng CAT6 UTP được sử dụng chất liệu CCA...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Hướng dẫn bảo hành', 'value': 'Xem ...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 212, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 32.650 ₫', 'title': '6 n...",NaN,NaN
3,8608925,8608925,"""6460064428402""","""Hộp mực dành cho máy in Laser Brother HL-L232...",hop-muc-may-in-laser-brother-hl-l2321d-p8608925,hop-muc-may-in-laser-brother-hl-l2321d-p860892...,https://tiki.vn/product-p8608925.html?spid=263...,simple,None,Hộp mực TN-2385/2325... hàng thay thế thích hợ...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '1 Th...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 212, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 107.900 ₫', 'title': '9 ...",NaN,NaN
4,74975883,74975883,"""8856697508143""","""Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép ...",bo-phat-wifi-tp-link-archer-c54-bang-tan-kep-c...,bo-phat-wifi-tp-link-archer-c54-bang-tan-kep-c...,https://tiki.vn/product-p74975883.html?spid=74...,simple,None,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '24 T...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 212, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 430.000 ₫', 'title': '5 ...",https://sand.tikicdn.com/ts/video/d3/14/01/473...,NaN


In [178]:
df_trans = df_product[['id', 'name', 'sku', 'price', 'list_price', 'discount', "discount_rate", "inventory_status"]]
df_trans.head()

,id,name,sku,price,list_price,discount,discount_rate,inventory_status
0,134561311,"""USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập ...","""8367091212771""",110000,110000,0,0,"""available"""
1,38323087,"""(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passp...","""9386902161867""",1385000,1390000,5000,0,"""available"""
2,26314426,"""Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu""","""9365508570174""",75000,75000,0,0,"""available"""
3,8608925,"""Hộp mực dành cho máy in Laser Brother HL-L232...","""6460064428402""",111900,111900,0,0,"""available"""
4,74975883,"""Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép ...","""8856697508143""",409000,858000,449000,52,"""available"""


In [198]:
# stock_item: max_sale_qty, qty
# quantity_sold: value
# brand: id, name
df_stock = df_product['stock_item'].apply(pd.Series)
df_qty = df_product['quantity_sold'].apply(pd.Series)
df_brand = df_product['brand'].apply(pd.Series)
df_stock = df_stock[['max_sale_qty', 'qty']]
df_qty = df_qty[['value']]
df_brand = df_brand[['id', 'name']]
df_qty.rename(columns={'value': 'qty_sold'}, inplace=True)
df_brand.rename(columns={'id': 'brand_id', 'name': 'brand_name'}, inplace=True)

In [199]:
df_final = pd.concat([df_trans, df_stock, df_qty, df_brand], axis=1)
df_final.head()

,id,name,sku,price,list_price,discount,discount_rate,inventory_status,max_sale_qty,qty,qty_sold,brand_id,brand_name
0,134561311,"""USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập ...","""8367091212771""",110000,110000,0,0,"""available""",1000.0,1000.0,9973,18845,SanDisk
1,38323087,"""(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passp...","""9386902161867""",1385000,1390000,5000,0,"""available""",1000.0,1000.0,2244,21862,WD
2,26314426,"""Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu""","""9365508570174""",75000,75000,0,0,"""available""",1000.0,1000.0,1120,111461,OEM
3,8608925,"""Hộp mực dành cho máy in Laser Brother HL-L232...","""6460064428402""",111900,111900,0,0,"""available""",1000.0,1000.0,2638,111461,OEM
4,74975883,"""Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép ...","""8856697508143""",409000,858000,449000,52,"""available""",1000.0,1000.0,4323,27464,TP-Link


In [155]:
df_final.to_csv("product.csv")